In [1]:
import os
import requests
from openpyxl import load_workbook
import xlrd
import csv
import pandas as pd
from bs4 import BeautifulSoup as bs
import re
from lxml import etree

In [91]:
filename = 'Lines in room 7 v4 copy.xlsx'
pdTest = pd.ExcelFile(filename)
pdWriter = pd.ExcelWriter(path=filename, engine='openpyxl')
pdWriter.book = load_workbook(filename)
pdWriter.sheets = dict((ws.title, ws) for ws in pdWriter.book.worksheets)

with open('OrderResults-802807-636834094995853281.xls') as xcl:
    t = xcl.read()
    s = bs(t, 'html.parser')
    p = s.body.findAll('table')[2:]
    for gtables in p:
        
        header_row = gtables.tr.text.split('\n')[3:-1] # Headers always in first table row
        strain = header_row[0]
        probes = header_row[1:]
        probes.insert(0,'ID')
        print(strain)
        
        strain_sheet = pdTest.parse(sheet_name=strain)
        header_list = {}
        [header_list.setdefault(col, None) for col in list(strain_sheet.columns)]
        
        for mouse in gtables.findAll('tr')[1:]:
            genotypes = mouse.text.split('\n')[3:-1]
            if len(genotypes) == len(header_row):
                
                genotypeDict = dict(zip(probes, genotypes))
                header_list.update(genotypeDict)
                strain_sheet = strain_sheet.append(header_list, ignore_index=True)
                strain_sheet.to_excel(pdWriter, sheet_name=strain,index=False)

pdWriter.save()

Grp KH288
Grp KH107
Sox10-SERT
Noggin
Grp-KH288 x tdTomato


In [78]:
filename = 'Lines in room 7 v4 copy.xlsx'
pdTest = pd.ExcelFile(filename)
pdWriter = pd.ExcelWriter(path=filename, engine='openpyxl')
pdWriter.book = load_workbook(filename)
pdWriter.sheets = dict((ws.title, ws) for ws in pdWriter.book.worksheets)

strain_sheet = pdTest.parse(sheet_name='Grp KH288')
header_list = {}
[header_list.setdefault(col, None) for col in list(strain_sheet.columns)]


[None, None, None, None, None, None, None, None, None, None, None, None]

In [ ]:

                strain_sheet = strain_sheet.append(header_list, ignore_index=True)
                strain_sheet.to_excel(pdWriter, sheet_name=strain,index=False)

In [79]:
header_list

{'TO DO:': None,
 'ID': None,
 'Genotype:': None,
 'CRE': None,
 'Strain': None,
 'DOB': None,
 'Age (Weeks)': None,
 'SEX': None,
 'Sire': None,
 'Dam': None,
 'Sac': None,
 'NOTES': None}

In [62]:
def getInfo(mouse, headerlist):

    headerlist['Strain'] = mouse.attrib['StrainName']
    headerlist['ID'] = mouse.attrib['Name']
    
    genotypes = mouse.getchildren()
    probeResults = {}
    [probeResults.setdefault(mouse.attrib['ProbeName'], mouse.attrib['Result']) for mouse in mouse]
    
    headerlist.update(probeResults)
    
    #print(header_list)

    return headerlist

In [65]:
filename = 'Lines in room 7 v4 copy.xlsx'
pdTest = pd.ExcelFile(filename)
pdWriter = pd.ExcelWriter(path=filename, engine='openpyxl')
pdWriter.book = load_workbook(filename)
pdWriter.sheets = dict((ws.title, ws) for ws in pdWriter.book.worksheets)

tree = etree.parse("OrderResults-802807.xml")
root = tree.getroot()
wellplate = root.getchildren()
prev_strain = None

for mice in wellplate[0]:
    
    strain = mice.attrib['StrainName']
    if prev_strain == None:
        print("New Strain: " + strain)
        prev_strain = strain
        strain_sheet = pdTest.parse(sheet_name=strain)
    elif prev_strain != strain:
        print("New Strain: " + strain + " Previous Strain: " + prev_strain)
        prev_strain_sheet = strain_sheet
        prev_strain_sheet.to_excel(pdWriter, sheet_name=prev_strain,index=False)
        pdWriter.save()
        strain_sheet = pdTest.parse(sheet_name=strain)
    
    header_list = {}
    [header_list.setdefault(col, None) for col in list(strain_sheet.columns)]
    
    header_list = getInfo(mice, header_list)

    strain_sheet = strain_sheet.append(header_list, ignore_index=True)

    prev_strain = strain

strain_sheet.to_excel(pdWriter, sheet_name=strain,index=False)
pdWriter.save()

New Strain: Grp KH288
New Strain: Grp KH107 Previous Strain: Grp KH288
New Strain: Sox10-SERT Previous Strain: Grp KH107
New Strain: Noggin Previous Strain: Sox10-SERT
New Strain: Grp-KH288 x tdTomato Previous Strain: Noggin


In [66]:
header_list

{'TO DO:': None,
 'ID': '45990.3',
 'Genotype:': None,
 'CRE': '-',
 'ROSA WT': '+',
 'STOP': None,
 'tdRFP': '+',
 'Strain': 'Grp-KH288 x tdTomato',
 'DOB': None,
 'Age (Weeks)': None,
 'SEX': None,
 'Sire': None,
 'Dam': None,
 'Sac': None,
 'NOTES': None,
 'Stop-3 Tg': '+'}

In [ ]:
Name ---> ID
StrainName ---> Strain
Result ---> Probe

In [35]:
len(wellplate[0])

57

In [40]:
wellplate[0][1].attrib

{'ID': '57767563', 'WellplateLocation': '2', 'Name': '45994.2', 'StrainName': 'Grp KH288'}

In [25]:
strain_sheet["ID"][12]

'45990.3'

In [67]:
strain_sheet

,TO DO:,ID,Genotype:,CRE,ROSA WT,STOP,tdRFP,Strain,DOB,Age (Weeks),SEX,Sire,Dam,Sac,NOTES,Stop-3 Tg
0,NaN,213313,NaN,+,+,-,-,NaN,NaT,6213.428571,NaN,NaN,NaN,NaN,Original Breeder - GrpKH288 (Not RFP tagged),NaN
1,NaN,45412.1,NaN,-,+,+,+,Grp-KH288/tdTomato,2018-12-11,7.000000,M,39766.2,213313.1,NaN,NaN,NaN
2,NaN,45412.2,Grp-KH288/tdTomato,+,+,+,+,Grp-KH288/tdTomato,2018-12-11,7.000000,M,39766.2,213313.1,NaN,NaN,NaN
3,NaN,45412.3,NaN,-,+,+,+,Grp-KH288/tdTomato,2018-12-11,7.000000,M,39766.2,213313.1,NaN,NaN,NaN
4,NaN,45989.1,Grp-KH288/tdTomato,+,+,+,+,Grp-KH288/tdTomato,2018-12-11,7.000000,F,39766.2,213313.1,NaN,NaN,NaN
5,NaN,45989.2,Grp-KH288/tdTomato,+,+,+,+,Grp-KH288/tdTomato,2018-12-11,7.000000,F,39766.2,213313.1,NaN,NaN,NaN
6,NaN,45989.3,NaN,-,+,+,+,Grp-KH288/tdTomato,2018-12-11,7.000000,F,39766.2,213313.1,NaN,NaN,NaN
7,NaN,45989.4,Grp-KH288/tdTomato,+,+,+,+,Grp-KH288/tdTomato,2018-12-11,7.000000,F,39766.2,213313.1,NaN,NaN,NaN
8,NaN,45990.1,Grp-KH288/tdTomato,+,+,+,+,Grp-KH288/tdTomato,2018-12-11,7.000000,F,39766.2,213313.1,NaN,NaN,NaN
9,NaN,45990.2,NaN,-,+,+,+,Grp-KH288/tdTomato,2018-12-11,7.000000,F,39766.2,213313.1,NaN,NaN,NaN
